In [1133]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_1.txt'
SEED = 9496461801973866405
# SEED = None

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)
tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
out_1, train_y

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


(tensor([0, 0, 0, 0, 0, 0, 0, 0]), tensor([1, 1, 1, 1, 0, 0, 0, 0]))

In [1134]:
Y = torch.randint(0,2, tm.l1.out.size())
tm.l1.out, Y, tm.l1.W

(tensor([[0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [1, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 1, 1]]),
 tensor([[0, 0, 1, 1, 0],
         [0, 1, 1, 0, 1],
         [1, 1, 0, 0, 1],
         [0, 0, 1, 0, 1],
         [0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [0, 1, 1, 1, 1]]),
 tensor([[1, 1, 1, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0],
         [0, 1, 1, 1, 0, 0]]))

In [1135]:
from tabulate import tabulate

tabular_W_collisions = [] # these are the necessary ones
for i in range(tm.l1.full_X.shape[0]):
    row_W_collisions = [[] for _ in range(tm.l1.in_dim * 2)]
    row_Y = Y[i]
    zero_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1).tolist()
    W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
    pos_W = W_halves[0]
    neg_W = W_halves[1]
    for pos_one_Y_idx in zero_Y_idxs:
        w_1 = pos_W[pos_one_Y_idx]
        prod = w_1 * neg_W[zero_Y_idxs]
        collision_idxs = (prod == 1).nonzero(as_tuple=False)
        for row_collision_idx, col_collision_idx in collision_idxs:
            neg_col_collision_idx = col_collision_idx + tm.l1.in_dim
            if pos_one_Y_idx not in row_W_collisions[col_collision_idx]:
                row_W_collisions[col_collision_idx].append(pos_one_Y_idx)
            if zero_Y_idxs[row_collision_idx] not in row_W_collisions[neg_col_collision_idx]:
                row_W_collisions[neg_col_collision_idx].append(zero_Y_idxs[row_collision_idx])

    tabular_W_collisions.append(row_W_collisions)

headers = [f"Column {i+1}" for i in range(len(tabular_W_collisions[0]))]
print(tabulate(tabular_W_collisions, headers=headers, tablefmt="fancy_grid"))

╒════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│ Column 1   │ Column 2   │ Column 3   │ Column 4   │ Column 5   │ Column 6   │
╞════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ []         │ []         │ [3]        │ []         │ []         │ [2]        │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [1]        │ [2, 4]     │ [1, 4]     │ [4]        │ [1]        │ [2]        │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [0, 1]     │ [0, 4]     │ []         │ [4]        │ [1]        │ []         │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ []         │ []         │ [4]        │ []         │ []         │ [2]        │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ []         │ []         │ [3]        │ []         │ []         │ [2]        │
├────────────┼────────────┼────────────┼

In [1136]:
from tabulate import tabulate

W_collisions = [[] for _ in range(tm.l1.W.shape[1])] # these are the necessary ones
for i in range(tm.l1.full_X.shape[0]):
    row_Y = Y[i]
    zero_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1).tolist()
    W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
    pos_W = W_halves[0]
    neg_W = W_halves[1]
    for pos_one_Y_idx in zero_Y_idxs:
        w_1 = pos_W[pos_one_Y_idx]
        prod = w_1 * neg_W[zero_Y_idxs]
        collision_idxs = (prod == 1).nonzero(as_tuple=False)
        for row_collision_idx, col_collision_idx in collision_idxs:
            neg_col_collision_idx = col_collision_idx + tm.l1.in_dim
            if pos_one_Y_idx not in W_collisions[col_collision_idx]:
                W_collisions[col_collision_idx].append(pos_one_Y_idx)
            if zero_Y_idxs[row_collision_idx] not in W_collisions[neg_col_collision_idx]:
                W_collisions[neg_col_collision_idx].append(zero_Y_idxs[row_collision_idx])

W_collisions

[[1, 0], [2, 4, 0, 3], [3, 1, 4], [4], [1], [2]]

In [1137]:
# W_deps_full = []
# for i, single_x in enumerate(tm.l1.full_X):
#     W_collisions_row = [[] for _ in range(tm.l1.in_dim * 2)]
#     row_Y = Y[i]
#     one_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1)
#     W_halves = torch.split(tm.l1.W, tm.l1.in_dim, dim=1)
#     pos_W = W_halves[0]
#     neg_W = W_halves[1]
#     for pos_one_Y_idx in one_Y_idxs:
#         w_1 = pos_W[pos_one_Y_idx]
#         for neg_one_Y_idx in one_Y_idxs:
#             w_2 = neg_W[neg_one_Y_idx]
#             pos_one_W_idxs = torch.nonzero(w_1 == 1).squeeze(1)
#             neg_one_W_idxs = torch.nonzero(w_2 == 1).squeeze(1)
#             for collision_idx in pos_one_W_idxs:
#                 if pos_one_Y_idx.item() not in W_collisions_row[collision_idx.item()]:
#                     W_collisions_row[collision_idx.item()].append(pos_one_Y_idx.item())
#             for collision_idx in neg_one_W_idxs:
#                 if neg_one_Y_idx.item() not in W_collisions_row[collision_idx.item() + tm.l1.in_dim]:
#                     W_collisions_row[collision_idx.item() + tm.l1.in_dim].append(neg_one_Y_idx.item())
#     W_deps_full.append(W_collisions_row)
# W_deps_full

In [1138]:
tm.l1.full_X

tensor([[1, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1],
        [1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0]])

In [1152]:
from tabulate import tabulate

flip_deps_for_ones_and_zeroes = []
mask_X = tm.l1.full_X == 1
for i, single_W in enumerate(tm.l1.W):
    col_Y = Y[:, i]
    zero_Y_idxs = torch.nonzero(col_Y == 1).squeeze(1)
    X_flip_row_idxs_by_col = [[] for _ in range(tm.l1.in_dim * 2)]

    for row in range(mask_X.shape[0]):
        for col in range(mask_X.shape[1]):
            if (not mask_X[row,col] and row in zero_Y_idxs):
                X_flip_row_idxs_by_col[col].append((row, 'one'))
            elif (mask_X[row,col] and row not in zero_Y_idxs):
                X_flip_row_idxs_by_col[col].append((row, 'zero'))

    flip_deps_for_ones_and_zeroes.append(X_flip_row_idxs_by_col)

headers = [f"Column {i+1}" for i in range(len(flip_deps_for_ones_and_zeroes[0]))]
print(tabulate(flip_deps_for_ones_and_zeroes, headers=headers, tablefmt="fancy_grid"))

╒═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════╤═════════════════════════════════════════════════════════════════════════════╕
│ Column 1                                                        │ Column 2                                                        │ Column 3                                                        │ Column 4                                                                    │ Column 5                                                                    │ Column 6                                                                    │
╞═════════════════════════════════════════════════════════════════╪═════════════════════════════════════════════════════════════════

In [1140]:
from tabulate import tabulate

mask_X = tm.l1.full_X == 1
flip_deps_for_ones = []
for i, single_W in enumerate(tm.l1.W):
    col_Y = Y[:, i]
    zero_Y_idxs = torch.nonzero(col_Y == 1).squeeze(1)
    X_flip_row_idxs_by_col = [[] for _ in range(tm.l1.in_dim * 2)]

    for row in range(mask_X.shape[0]):
        for col in range(mask_X.shape[1]):
            if (not mask_X[row,col] and row in zero_Y_idxs):
                X_flip_row_idxs_by_col[col].append(row)

    flip_deps_for_ones.append(X_flip_row_idxs_by_col)

headers = [f"Column {i+1}" for i in range(len(flip_deps_for_ones[0]))]
print(tabulate(flip_deps_for_ones, headers=headers, tablefmt="fancy_grid"))

╒════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│ Column 1   │ Column 2   │ Column 3   │ Column 4   │ Column 5   │ Column 6   │
╞════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ [6]        │ [6]        │ [2]        │ [2]        │ [2]        │ [6]        │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [6, 7]     │ [1, 6]     │ [2]        │ [1, 2]     │ [2, 7]     │ [1, 6, 7]  │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [4, 7]     │ [0, 1, 4]  │ [0, 4]     │ [0, 1, 3]  │ [3, 7]     │ [1, 3, 7]  │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [4, 7]     │ [0, 4]     │ [0, 4]     │ [0]        │ [7]        │ [7]        │
├────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ [7]        │ [1]        │ [2]        │ [1, 2, 3]  │ [2, 3, 7]  │ [1, 3, 7]  │
╘════════════╧════════════╧════════════╧

In [1141]:
from tabulate import tabulate

one_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = Y[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1).tolist()
    one_Y_row_idxs_per_W_row.append(zero_Y_idxs)
one_Y_row_idxs_per_W_row

[[2, 6], [1, 2, 6, 7], [0, 1, 3, 4, 7], [0, 4, 7], [1, 2, 3, 7]]

In [1142]:
W_collisions

[[1, 0], [2, 4, 0, 3], [3, 1, 4], [4], [1], [2]]

In [1143]:
SEED = 9496461801973866405
if SEED:
    random.seed(SEED)
    torch.manual_seed(SEED)
else:
    seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
    random.seed(seed)
    torch.manual_seed(seed)
    print(seed)

def is_there_conflict(col_idx, row_idx, chosen_col_idxs):
    neg_col_idx =  (col_idx + tm.l1.in_dim) % (tm.l1.in_dim * 2)
    if neg_col_idx in chosen_col_idxs:
        return len(set(one_Y_row_idxs_per_W_row[row_idx]) & set(one_Y_row_idxs_per_W_row[row_idx])) > 0
    return False

def can_col_idx_be_used(col_idx, row_idx, cannot_be_changed):
    return col_idx in cannot_be_changed[row_idx] if row_idx in cannot_be_changed else False

def func(W_collisions):
    new_W = torch.zeros_like(tm.l1.W)
    
    cannot_be_changed = {}
    for i in range(len(W_collisions)//2):
        if len(W_collisions[i]) > 0:
            neg_i = i + len(W_collisions)//2
            selected_index = random.choice([i,neg_i])
            if len(W_collisions[i]) > len(W_collisions[neg_i]):
                selected_index = neg_i
            elif len(W_collisions[i]) < len(W_collisions[neg_i]):
                selected_index = i
            
            for row_i in W_collisions[selected_index]:
                if row_i not in cannot_be_changed:
                    cannot_be_changed[row_i] = []
                cannot_be_changed[row_i].append(selected_index)

    selected_col_idxs = torch.zeros(new_W.shape[0]).tolist()
    remaining_row_idxs = [i for i in range(new_W.shape[0]) if i not in cannot_be_changed.keys()]
    for row_idx in (sorted(list(cannot_be_changed.keys())) + remaining_row_idxs):
        available_col_idxs = [i for i in range(new_W.shape[1])]
        random.shuffle(available_col_idxs)
        candidate_list_idx = 0
        candidate_col_idx = available_col_idxs[candidate_list_idx]
        col_idx_cannot_be_used = can_col_idx_be_used(candidate_col_idx, row_idx, cannot_be_changed)
        while col_idx_cannot_be_used or is_there_conflict(candidate_col_idx, row_idx, selected_col_idxs):
            candidate_list_idx += 1
            candidate_col_idx = available_col_idxs[candidate_list_idx]
            col_idx_cannot_be_used = can_col_idx_be_used(candidate_col_idx, row_idx, cannot_be_changed)

        new_W[row_idx, candidate_col_idx] = 1
        selected_col_idxs[row_idx] = candidate_col_idx
            
    return selected_col_idxs, new_W

selected_W_col_idx_per_W_row, new_W = func(W_collisions)
selected_W_col_idx_per_W_row, new_W

([0, 2, 4, 0, 2],
 tensor([[1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0]]))

In [1144]:
new_full_X = torch.clone(tm.l1.full_X)
mask_X = torch.zeros_like(new_full_X)
for row_idx, col_indx in enumerate(selected_W_col_idx_per_W_row):
    X_row_idxs = flip_deps_for_ones[row_idx][col_indx]
    for X_row_idx in X_row_idxs:
        mask_X[X_row_idx, col_indx] = 1
        neg_index = (col_indx + tm.l1.in_dim) % (tm.l1.in_dim*2)
        mask_X[X_row_idx, neg_index] = 1

new_full_X[mask_X.bool()] = 1 - new_full_X[mask_X.bool()]
mask_X,new_full_X

(tensor([[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 1],
         [0, 1, 0, 0, 1, 0],
         [1, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0],
         [1, 1, 0, 1, 1, 0]]),
 tensor([[1, 0, 0, 0, 1, 1],
         [1, 0, 1, 0, 1, 0],
         [1, 1, 1, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [1, 0, 0, 0, 1, 1],
         [0, 1, 0, 1, 0, 1],
         [1, 0, 1, 0, 1, 0],
         [1, 0, 1, 0, 1, 0]]))

In [1148]:
from tsetlin import TsetlinBase

tb = TsetlinBase()
y2 = tb.conjunction_mul(new_full_X.unsqueeze(1), new_W)
y2

tensor([[1, 0, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 1, 1, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])

In [1146]:
from tabulate import tabulate

mask_X = new_full_X == 1
flip_deps_for_zeroes = []
for i, single_W in enumerate(tm.l1.W):
    col_Y = Y[:, i]
    zero_Y_idxs = torch.nonzero(col_Y == 0).squeeze(1)
    X_flip_row_idxs_by_col = [[] for _ in range(tm.l1.in_dim * 2)]

    for row in range(mask_X.shape[0]):
        for col in range(mask_X.shape[1]):
            if (mask_X[row,col] and row in zero_Y_idxs):
                X_flip_row_idxs_by_col[col].append(row)

    flip_deps_for_zeroes.append(X_flip_row_idxs_by_col)

headers = [f"Column {i+1}" for i in range(len(flip_deps_for_zeroes[0]))]
print(tabulate(flip_deps_for_zeroes, headers=headers, tablefmt="fancy_grid"))

╒═════════════════╤════════════╤══════════════╤════════════╤═════════════════╤════════════╕
│ Column 1        │ Column 2   │ Column 3     │ Column 4   │ Column 5        │ Column 6   │
╞═════════════════╪════════════╪══════════════╪════════════╪═════════════════╪════════════╡
│ [0, 1, 3, 4, 7] │ [5]        │ [1, 3, 7]    │ [5]        │ [0, 1, 3, 4, 7] │ [0, 4, 5]  │
├─────────────────┼────────────┼──────────────┼────────────┼─────────────────┼────────────┤
│ [0, 3, 4]       │ [5]        │ [3]          │ [5]        │ [0, 3, 4]       │ [0, 4, 5]  │
├─────────────────┼────────────┼──────────────┼────────────┼─────────────────┼────────────┤
│ [2, 6]          │ [2, 5]     │ [2, 6]       │ [5]        │ [6]             │ [5]        │
├─────────────────┼────────────┼──────────────┼────────────┼─────────────────┼────────────┤
│ [1, 2, 3, 6]    │ [2, 5]     │ [1, 2, 3, 6] │ [5]        │ [1, 3, 6]       │ [5]        │
├─────────────────┼────────────┼──────────────┼────────────┼─────────────────┼──

In [1147]:
from tabulate import tabulate

zero_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = Y[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 0).squeeze(1).tolist()
    zero_Y_row_idxs_per_W_row.append(zero_Y_idxs)
zero_Y_row_idxs_per_W_row

[[0, 1, 3, 4, 5, 7], [0, 3, 4, 5], [2, 5, 6], [1, 2, 3, 5, 6], [0, 4, 5, 6]]

In [1159]:
one_Y_row_idxs_per_W_row, zero_Y_row_idxs_per_W_row

([[2, 6], [1, 2, 6, 7], [0, 1, 3, 4, 7], [0, 4, 7], [1, 2, 3, 7]],
 [[0, 1, 3, 4, 5, 7], [0, 3, 4, 5], [2, 5, 6], [1, 2, 3, 5, 6], [0, 4, 5, 6]])

In [1404]:
SEED = 9496461801973866405
# SEED = None

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

random.seed(SEED)
torch.manual_seed(SEED)

tm = TsetlinMachine(train_x.shape[1], 5)
out_1 = tm.forward(train_x)
tm.l1.out

tensor([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1]])

In [1405]:
from tabulate import tabulate

zero_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = tm.l1.out[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 0).squeeze(1).tolist()
    zero_Y_row_idxs_per_W_row.append(zero_Y_idxs)
zero_Y_row_idxs_per_W_row

[[0, 1, 2, 4, 5, 6, 7],
 [0, 2, 3, 4, 5, 6, 7],
 [0, 1, 3, 4, 6, 7],
 [0, 1, 2, 4, 5, 6],
 [0, 1, 2, 3, 4, 5, 6]]

In [1406]:
from tabulate import tabulate

one_Y_row_idxs_per_W_row = []
for i in range(tm.l1.W.shape[0]):
    row_Y = tm.l1.out[:, i]
    zero_Y_idxs = torch.nonzero(row_Y == 1).squeeze(1).tolist()
    one_Y_row_idxs_per_W_row.append(zero_Y_idxs)
one_Y_row_idxs_per_W_row

[[3], [1], [2, 5], [3, 7], [7]]

In [1407]:
tm.l1.W, tm.l1.full_X

(tensor([[1, 1, 1, 0, 0, 0],
         [1, 0, 1, 0, 1, 0],
         [0, 1, 0, 0, 0, 1],
         [0, 1, 1, 0, 0, 0],
         [0, 1, 1, 1, 0, 0]]),
 tensor([[1, 0, 0, 0, 1, 1],
         [1, 0, 1, 0, 1, 0],
         [1, 1, 0, 0, 0, 1],
         [1, 1, 1, 0, 0, 0],
         [0, 0, 0, 1, 1, 1],
         [0, 1, 0, 1, 0, 1],
         [0, 0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0, 0]]))

In [1408]:
import itertools

def all_subsets(input_set):
    s = list(input_set)
    return [set(combination) for r in range(1, len(s) + 1) for combination in itertools.combinations(s, r)]

set_zeroes = [set(x) for x in zero_Y_row_idxs_per_W_row]

neg_intersection = None
for x in set_zeroes:
    if neg_intersection is None:
        neg_intersection = x
    else:
        neg_intersection = neg_intersection & x

updated_set_zeroes = [ (x - neg_intersection) for x in set_zeroes if x != neg_intersection]
print(updated_set_zeroes)
for i in range((tm.l1.W.shape[1]//2)-1):
    all_zero_rows = set().union(*updated_set_zeroes)
    all_possible_inteserctions = all_subsets(all_zero_rows)
    intersection_sets = {}
    for set_x in updated_set_zeroes:
        for intersection in all_possible_inteserctions:
            if intersection.issubset(set_x):
                intersection_t = tuple(intersection)
                if intersection_t not in intersection_sets:
                    intersection_sets[intersection_t] = 0
                intersection_sets[intersection_t] += 1

    highest_intersection_set = None
    highest_interesection_count = None
    for k,v in intersection_sets.items():
        if highest_intersection_set is None or v > highest_interesection_count:
            highest_interesection_count = v
            highest_intersection_set = k
        elif v == highest_interesection_count and len(k) > len(highest_intersection_set):
            highest_interesection_count = v
            highest_intersection_set = k
    print(highest_interesection_count)
    print(highest_intersection_set)
    updated_set_zeroes = [(x - set(highest_intersection_set)) for x in updated_set_zeroes if x != set(highest_intersection_set)]
    print(updated_set_zeroes)

highest_intersection_set, updated_set_zeroes

[{1, 2, 5, 7}, {2, 3, 5, 7}, {1, 3, 7}, {1, 2, 5}, {1, 2, 3, 5}]
4
(2, 5)
[{1, 7}, {3, 7}, {1, 3, 7}, {1}, {1, 3}]
4
(1,)
[{7}, {3, 7}, {3, 7}, {3}]


((1,), [{7}, {3, 7}, {3, 7}, {3}])

In [1409]:
one_sets = [set(x) for x in one_Y_row_idxs_per_W_row]
zero_sets = [set(x) for x in zero_Y_row_idxs_per_W_row]
tracking = {tuple(x):y for x, y in zip(one_sets, zero_sets)}
solution = {tuple(x):[0,0,0] for x in one_sets}

cols = []
for i in range(tm.l1.in_dim):
    one_sets = []
    zero_sets = []
    for k,v in tracking.items():
        one_sets.append(set(k))
        zero_sets.append(v)

    one_sets_occurrence_in_zeros = {tuple(x):[] for x in one_sets}

    for x in one_sets:
        for i, y in enumerate(zero_sets):
            if x.issubset(y):
                one_sets_occurrence_in_zeros[tuple(x)].append(i)

    max_occurrence_term = None
    max_occurrence_count = None
    count_dict = {k: len(v) for k,v in one_sets_occurrence_in_zeros.items()}
    for k,v in count_dict.items():
        if max_occurrence_term is None or v > max_occurrence_count:
            max_occurrence_count = v
            max_occurrence_term = k
        elif v == max_occurrence_count and len(k) > len(max_occurrence_term):
            max_occurrence_count = v
            max_occurrence_term = k

    one_rows = set(max_occurrence_term)
    corresponding_zeroes = tracking[max_occurrence_term]
    zero_rows = set().union(*[ x for i, x in enumerate(one_sets) if i in one_sets_occurrence_in_zeros[max_occurrence_term]])
    
    match_col = -1
    match_one_rows_side = None
    max_zero_rows_intersection = -1
    for i, col in enumerate(cols):
        # TODO: it doesnt have to be an exact subset, but something that could be made into a subset
        if one_rows.issubset(col[0]) and len(corresponding_zeroes & col[0]) == 0:
            if match_col == -1 or len(zero_rows & col[1]) > max_zero_rows_intersection:
                match_col = i
                max_zero_rows_intersection = len(zero_rows & col[1])
                match_one_rows_side = 0
        # elif one_rows.issubset(col[1]) and len(corresponding_zeroes & col[1]) == 0:
        #     if match_col == -1 or len(zero_rows & col[0]) > max_zero_rows_intersection:
        #         match_col = i
        #         max_zero_rows_intersection = len(zero_rows & col[0])
        #         match_one_rows_side = 1

    if match_col == -1:
        all_zero_rows = zero_rows
        to_add = [one_rows, all_zero_rows]
        left_side = one_rows
        right_side = all_zero_rows
        cols.append(to_add)
        match_col = len(cols) - 1
        match_one_rows_side = 0
    else:
        found_col = cols[match_col]
        # combined_zeroes = corresponding_zeroes | zero_rows
        allowed_zeroes = zero_rows - found_col[match_one_rows_side]

        all_zero_rows = found_col[1- match_one_rows_side] | allowed_zeroes
        found_col[1- match_one_rows_side] = all_zero_rows

        left_side = found_col[0]
        right_size = found_col[1]
        
    remove_ks = []
    for k,v in tracking.items():
        k_set = set(k)
        if k_set.issubset(left_side):
            sub = v - right_side
            if len(sub) > 0:
                tracking[k] = sub
                solution[tuple(k_set)][match_col] += 1
            else:
                remove_ks.append(k)
                solution[tuple(k_set)][match_col] += 1
        elif k_set.issubset(right_side):
            sub = v - left_side
            if len(sub) > 0:
                solution[tuple(k_set)][match_col] -= 1
                tracking[k] = sub
            else:
                remove_ks.append(k)
                solution[tuple(k_set)][match_col] -= 1
    for k in remove_ks:
        tracking.pop(k)

cols, solution, tracking

([[{2, 5}, {1, 3, 7}], [{1}, {3, 7}], [{3}, {7}]],
 {(3,): [-1, -1, 1],
  (1,): [-1, 1, 0],
  (2, 5): [1, 0, 0],
  (3, 7): [-1, -1, 0],
  (7,): [-1, -1, -1]},
 {(3,): {0, 4, 6},
  (1,): {0, 4, 6},
  (2, 5): {0, 4, 6},
  (3, 7): {0, 4, 6},
  (7,): {0, 4, 6}})

In [1410]:
one_row_to_col_idxs = {}
available_rows_per_col = []
for col in cols:
    all_rows = col[0] | col[1]
    remaining_rows = set(range(tm.l1.full_X.shape[0])) - all_rows
    available_rows_per_col.append(remaining_rows)


for x in one_Y_row_idxs_per_W_row:
    for i, col in enumerate(cols):
        if set(x).issubset(col[0]):
            if tuple(x) not in one_row_to_col_idxs:
                one_row_to_col_idxs[tuple(x)] = []
            one_row_to_col_idxs[tuple(x)].append((i,0))

        elif set(x).issubset(col[1]):
            if tuple(x) not in one_row_to_col_idxs:
                one_row_to_col_idxs[tuple(x)] = []
            one_row_to_col_idxs[tuple(x)].append((i,1))

col_idxs_to_one_rows = {}
for k,v in one_row_to_col_idxs.items():
    for col_idx in v:
        if col_idx not in col_idxs_to_one_rows:
            col_idxs_to_one_rows[col_idx] = []
        col_idxs_to_one_rows[col_idx].append(k)

sorted_keys = sorted(col_idxs_to_one_rows, key=lambda x: len(col_idxs_to_one_rows[x]), reverse=True)
for key in sorted_keys:
    one_rows = [x for x in col_idxs_to_one_rows[key] if x in tracking]
    if one_rows:

        intersect = set(range(tm.l1.full_X.shape[0])).intersection(*[tracking[x] for x in one_rows])
        intersect_with_available = intersect & available_rows_per_col[key[0]]

        available_rows_per_col[key[0]] -= intersect_with_available
        col_part = cols[key[0]][1-key[1]]
        col_part |= intersect_with_available
        remove_ks = []
        for k,v in tracking.items():
            if k in one_rows:
                sub = v - intersect_with_available
                if len(sub) > 0:
                    tracking[k] = sub
                else:
                    remove_ks.append(k)

        for k in remove_ks:
            tracking.pop(k)

remove_ks = []
for k,v in tracking.items():
    for i, col in enumerate(cols):
        col_remaining_size = tm.l1.full_X.shape[0]-(len(col[0]) + len(col[1]))
        if len(k) <= col_remaining_size:
            if not set(k).issubset(col[0]) and len(v & col[0]) == 0:
                col[0] |= set(k)
                col_remaining_size -= len(k)
                remaining_v = list(v - col[1])
                remaining_to_add = remaining_v[:min(col_remaining_size, len(remaining_v))]
                remaining_v_size = len(remaining_v) - col_remaining_size
                col[1] |= set(remaining_to_add)
                if remaining_v_size > 0:
                    tracking[k] = sub
                    solution[k][i] += 1
                else:
                    remove_ks.append(k)
                    solution[k][i] += 1
                break
            elif not set(k).issubset(col[1]) and len(v & col[1]) == 0:
                col[1] |= set(k)
                col_remaining_size -= len(k)
                remaining_v = list(v - col[0])
                remaining_to_add = remaining_v[:min(col_remaining_size, len(remaining_v))]
                remaining_v_size = len(remaining_v) - col_remaining_size
                col[0] |= set(remaining_to_add)
                if remaining_v_size > 0:
                    tracking[k] = sub
                    solution[k][i] -= 1
                else:
                    remove_ks.append(k)
                    solution[k][i] -= 1
                break
for k in remove_ks:
    tracking.pop(k)
cols, tracking, solution

([[{0, 2, 4, 5, 6}, {1, 3, 7}], [{1, 2, 5}, {0, 3, 4, 6, 7}], [{3}, {7}]],
 {},
 {(3,): [-1, -1, 1],
  (1,): [-1, 1, 0],
  (2, 5): [1, 1, 0],
  (3, 7): [-1, -1, 0],
  (7,): [-1, -1, -1]})

In [1411]:
for col in cols:
    remaining_rows = set(range(tm.l1.full_X.shape[0])) - (col[0] | col[1])
    col[1] |= remaining_rows
cols

[[{0, 2, 4, 5, 6}, {1, 3, 7}],
 [{1, 2, 5}, {0, 3, 4, 6, 7}],
 [{3}, {0, 1, 2, 4, 5, 6, 7}]]

In [1412]:
new_full_X = torch.zeros_like(tm.l1.full_X)
for i, col in enumerate(cols):
    new_full_X[list(col[0]), i] = 1
    new_full_X[list(col[1]), i + tm.l1.in_dim] = 1
new_full_X


tensor([[1, 0, 0, 0, 1, 1],
        [0, 1, 0, 1, 0, 1],
        [1, 1, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0],
        [1, 0, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1]])

In [1413]:
new_W = torch.zeros_like(tm.l1.W)
for i, x in enumerate(one_Y_row_idxs_per_W_row):
    W_row_values = solution[tuple(x)]
    for j, val in enumerate(W_row_values):
        if val > 0:
            new_W[i, j] = 1
        elif val < 0:
            new_W[i, j + tm.l1.in_dim] = 1
new_W

tensor([[0, 0, 1, 1, 1, 0],
        [0, 1, 0, 1, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1]])

In [1414]:
from tsetlin import TsetlinBase

tb = TsetlinBase()
y2 = tb.conjunction_mul(new_full_X.unsqueeze(1), new_W)
y2

tensor([[0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 1, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1]])